In [1]:
import csv
import pandas as pd

In [2]:
from sklearn import linear_model
import numpy as np

In [3]:
from itertools import chain

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [4]:
from nltk.corpus import stopwords
from nltk.corpus import names
import string
import re
from nltk import FreqDist
from Gensim import corpora, models, similarities, utils

ImportError: No module named Gensim

In [4]:
sw = set(stopwords.words('english'))
punctuation = set(string.punctuation)
all_names = set([name.lower() for name in names.words()])

In [5]:
cd /Users/tianhaolu/Documents/Playground/

/Users/tianhaolu/Documents/Playground


In [6]:
csv_file_object = csv.reader(open('/Users/tianhaolu/Documents/Playground/all_reuters_article_info.csv')) 

In [22]:
negative=csv.reader(open('/Users/tianhaolu/Documents/Playground/all_reuters_matched_articles_filtered.csv')) 

In [28]:
positive=pd.read_csv("all_reuters_article_info.csv")
negative=pd.read_csv("all_reuters_matched_articles_filtered.csv")

In [33]:
negative_0=negative[['PubMed_ID', 'citation_title', 'journal', 'authors', 'affiliation', 'abstract', 'MeSH terms']]

In [29]:
positive.columns=['PubMed_ID', 'citation_title', 'journal', 'authors', 'affiliation', 'abstract', 'MeSH terms']
negative.columns=['matched_ID','PubMed_ID', 'citation_title', 'journal', 'authors', 'affiliation', 'abstract', 'MeSH terms']

In [7]:
def isStopWord(word):
    return (word in sw or word in punctuation) or not word.isalpha() or word in all_names

In [31]:
Object = []
for row in csv_file_object:
    Object.append(row[5])

In [30]:
Object = []
for row in positive:
    Object.append(row[5])

In [32]:
List = []
for obj in Object:
    temp = [ ]
    for flag in obj.split(" "):
        if not isStopWord(flag.lower()) and len(flag) > 1:
            temp.append(flag)
    List.append(temp)

In [10]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
NewList = []
for L in List:
    temp = []
    for l in L:
        temp.append(st.stem(l))
        ll = " ".join(temp)
    NewList.append(ll)

In [11]:
flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]

In [12]:
newList = flatten(NewList)

In [13]:
words = FreqDist(newList)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
vectorizer = TfidfVectorizer(stop_words='english')

In [17]:
matrix = vectorizer.fit_transform(NewList)

In [18]:
matrix

<1431x6319 sparse matrix of type '<type 'numpy.float64'>'
	with 88440 stored elements in Compressed Sparse Row format>

In [19]:
import numpy as np

In [20]:
sums = np.array(matrix.sum(axis=0)).ravel()

In [21]:
sums

array([ 1.60442668,  0.5612261 ,  0.10858773, ...,  0.31539912,
        0.54936828,  0.0880355 ])

In [39]:
positive['published']=1
negative_0['published']=0

In [40]:
train=pd.concat([positive,negative_0])

In [41]:
train.shape

(29788, 8)

In [46]:
train.drop_duplicates().shape

(12568, 8)

In [49]:
train_dedup=train.drop_duplicates()

In [71]:
train_dedup1=train_dedup.reset_index()

In [76]:
train_dedup1['authors'][0].split('and')

['Miro F ', ' Vidal E ', ' Balasch J']

In [77]:
def author_to_list(x):
    authors=x.split('and')
    return authors

In [79]:
train_dedup1['Author']=map(lambda x: author_to_list(x), train_dedup1['authors'])

In [80]:
train_dedup1

,index,PubMed_ID,citation_title,journal,authors,affiliation,abstract,MeSH terms,published,Author
0,0,22183210,Increased live birth rate in twin pregnancies ...,Obstet Gynecol,Miro F and Vidal E and Balasch J,Missing,OBJECTIVE: To estimate why the live birth rate...,Embryo Loss/epidemiology *Embryonic Developmen...,1,"[Miro F , Vidal E , Balasch J]"
1,1,22170364,Beef in an Optimal Lean Diet study: effects on...,Am J Clin Nutr,Roussell MA and Hill AM and Gaugler TL and Wes...,Missing,BACKGROUND: A Step I diet with lean beef compa...,Animals Apolipoproteins/*blood C-Reactive Prot...,1,"[Roussell MA , Hill AM , Gaugler TL , West ..."
2,2,22176841,Physical activity and other health-risk behavi...,Am J Prev Med,Kwan MY and Cairney J and Faulkner GE and Pull...,Missing,BACKGROUND: Research consistently demonstrates...,Adolescent Adult Age Factors Alcohol Drinking/...,1,"[Kwan MY , Cairney J , Faulkner GE , Pullen..."
3,3,22193170,Is illicit drug use harmful to cognitive funct...,Am J Epidemiol,Dregan A and Gulliford MC,Missing,"From March to July of 2011, the authors invest...",Adult Cognition/*drug effects Humans Middle Ag...,1,"[Dregan A , Gulliford MC]"
4,4,22213750,Physical activity and performance at school: a...,Arch Pediatr Adolesc Med,Singh A and Uijtdewilligen L and Twisk JW and ...,vumc,OBJECTIVE: To describe the prospective relatio...,*Achievement Adolescent Child *Cognition Exerc...,1,"[Singh A , Uijtdewilligen L , Twisk JW , va..."
5,5,22170362,Isoflavone intake and risk of gastric cancer: ...,Am J Clin Nutr,Hara A and Sasazuki S and Inoue M and Iwasaki ...,Missing,BACKGROUND: Isoflavones are structurally simil...,Aged *Diet Diet Surveys Estrogen Replacement T...,1,"[Hara A , Sasazuki S , Inoue M , Iwasaki M ..."
6,6,22196775,Carotid intimal-medial thickness in active pro...,Am J Cardiol,Hurst RT and Nelson MR and Kendall CB and Cha ...,mayo,Risk of cardiovascular disease and death in re...,Adult Cardiovascular Diseases/*epidemiology/et...,1,"[Hurst RT , Nelson MR , Kendall CB , Cha SS..."
7,7,22168350,Does minimum pricing reduce alcohol consumptio...,Addiction,Stockwell T and Auld MC and Zhao J and Martin G,Missing,AIMS: Minimum alcohol prices in British Columb...,Adolescent Adult Aged Alcohol Drinking/*econom...,1,"[Stockwell T , Auld MC , Zhao J , Martin G]"
8,8,22215166,Bariatric surgery and long-term cardiovascular...,JAMA,Sjostrom L and Peltonen M and Jacobson P and S...,Missing,CONTEXT: Obesity is a risk factor for cardiova...,Adult *Bariatric Surgery Cardiovascular Diseas...,1,"[Sjostrom L , Peltonen M , Jacobson P , Sjo..."
9,9,22217938,Vaccine protection against acquisition of neut...,Nature,Barouch DH and Liu J and Li H and Maxfield LF ...,harvard,Preclinical studies of human immunodeficiency ...,AIDS Vaccines/immunology Adenoviridae/genetics...,1,"[Barouch DH , Liu J , Li H , Maxfield LF , ..."


In [81]:
vectorizer.vocabulary_

{u'opensigl': 3944,
 u'orthogon': 3966,
 u'screenees': 5036,
 u'yellow': 6298,
 u'circuitry': 955,
 u'bisphenol': 599,
 u'psory': 4588,
 u'authorit': 440,
 u'buddy': 712,
 u'aug': 432,
 u'swab': 5562,
 u'insign': 2747,
 u'alswh': 194,
 u'selen': 5068,
 u'chimpanz': 908,
 u'cytosol': 1302,
 u'aut': 438,
 u'accum': 49,
 u'maladapt': 3210,
 u'downstream': 1615,
 u'caffein': 744,
 u'dutch': 1656,
 u'voic': 6156,
 u'lumin': 3163,
 u'delt': 1400,
 u'govern': 2282,
 u'uncertain': 5900,
 u'lidocain': 3072,
 u'vast': 6076,
 u'disturb': 1575,
 u'cedpv': 843,
 u'scholar': 5017,
 u'upd': 6021,
 u'unscreen': 6001,
 u'propens': 4542,
 u'tak': 5607,
 u'cholesterol': 921,
 u'markov': 3242,
 u'bedaquilin': 522,
 u'second': 5053,
 u'aggreg': 139,
 u'prognos': 4523,
 u'neg': 3593,
 u'altitud': 197,
 u'asid': 367,
 u'lightn': 3085,
 u'transcrib': 5785,
 u'vest': 6103,
 u'nev': 3653,
 u'new': 3654,
 u'databank': 1317,
 u'succumb': 5507,
 u'nep': 3608,
 u'pandem': 4049,
 u'ongo': 3934,
 u'elimin': 1746,
 u'

In [85]:
matrix.shape

(1431, 6319)

In [86]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import csv
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import vstack


label2 = pd.read_csv("all_reuters_matched_articles_filtered.csv", header=None)
label1 = pd.read_csv("all_reuters_article_info.csv", header=None)
label2 = label2.drop([0,1], axis=1)
label1 = label1.drop([0], axis=1)

label1.columns = [1,2,3,4,5,6]
label2.columns = [1,2,3,4,5,6]
#add label
label1['label'] = 1
label2['label'] = 0

data = pd.concat([label1,label2], ignore_index=True)
data = data.drop_duplicates()


def isStopWord(word):
	return (word in sw or word in punctuation) or not word.isalpha() or word in all_names


sw = set(stopwords.words('english'))
punctuation = set(string.punctuation)
all_names = set([name.lower() for name in names.words()])

List = []
for obj in data[6]:
    temp = [ ]
    for flag in obj.split(" "):
        if not isStopWord(flag.lower()) and len(flag) > 1:
            temp.append(flag)
    List.append(temp)


st = LancasterStemmer()
NewList = []
for L in List:
    temp = []
    for l in L:
        temp.append(st.stem(l))
        ll = " ".join(temp)
    NewList.append(ll)

vectorizer = TfidfVectorizer(stop_words='english')
MeSH_terms = vectorizer.fit_transform(NewList)
MeSH_terms = MeSH_terms.todense()
 

In [99]:
MeSH_terms.shape

(12565, 3595)

In [102]:
type(MeSH_terms)

numpy.matrixlib.defmatrix.matrix

In [100]:
list(vectorizer.vocabulary_)

[u'disp',
 u'reductas',
 u'splicing',
 u'phosphoryl',
 u'child',
 u'yellow',
 u'derm',
 u'sleep',
 u'nocard',
 u'dist',
 u'galactosid',
 u'anticholesterem',
 u'valvuloplasty',
 u'spin',
 u'intron',
 u'selen',
 u'hybridoma',
 u'granulocyt',
 u'urinalys',
 u'aut',
 u'postmortem',
 u'glomerul',
 u'spec',
 u'endothel',
 u'digit',
 u'risk',
 u'spodopter',
 u'voic',
 u'delph',
 u'microtomograph',
 u'condylomat',
 u'govern',
 u'neut',
 u'affect',
 u'vill',
 u'quantit',
 u'neur',
 u'biometry',
 u'osteochondrit',
 u'inbr',
 u'school',
 u'nanowir',
 u'cion',
 u'hepatocyt',
 u'propens',
 u'curricul',
 u'cholesterol',
 u'el',
 u'linsee',
 u'markov',
 u'marijuan',
 u'eustach',
 u'reimburs',
 u'craniotom',
 u'direct',
 u'bicyclo',
 u'horn',
 u'pneum',
 u'second',
 u'street',
 u'aggreg',
 u'prognos',
 u'chel',
 u'chem',
 u'bicyc',
 u'natriuret',
 u'nef',
 u'neg',
 u'altitud',
 u'nee',
 u'neb',
 u'transcrib',
 u'poison',
 u'saimir',
 u'monoest',
 u'fossil',
 u'nev',
 u'new',
 u'carp',
 u'nep',
 u'pand

In [97]:
data[list(vectorizer.vocabulary_)]=MeSH_terms

KeyError: "[u'disp' u'reductas' u'splicing' ..., u'kain' u'emerg' u'rep'] not in index"

In [104]:
terms_tfidf=pd.DataFrame(MeSH_terms,columns=list(vectorizer.vocabulary_))

In [105]:
terms_tfidf.head()

,disp,reductas,splicing,phosphoryl,child,yellow,derm,sleep,nocard,dist,...,desoxycorticosteron,peptid,infarct,xenop,bcg,fung,ophthalmolog,kain,emerg,rep
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
tags = [
  "python, tools",
  "linux, tools, ubuntu",
  "distributed systems, linux, networking, tools",
]

In [109]:
from sklearn.feature_extraction.text import CountVectorizer

In [107]:
vec = CountVectorizer(tokenizer=tokenize)
data = vec.fit_transform(tags).toarray()
print data

NameError: name 'tokenize' is not defined

In [113]:
cv = CountVectorizer(vocabulary=['hot', 'cold', 'old'])

In [114]:
cv.fit_transform(['pease porridge hot', 'pease porridge cold', 'pease porridge in the pot', 'nine days old']).toarray()

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 1]])

In [241]:
def author_to_list(x):
    authors=x.split(' and ')
    return authors

data['Author']=map(lambda x: author_to_list(x), data[3])

In [243]:
data1

,index,1,2,3,4,5,6,label,Author
0,0,Eighty-year trends in infant weight and length...,J Pediatr,Johnson W and Choh AC and Soloway LE and Czerw...,Missing,OBJECTIVES: To investigate secular trends in w...,Analysis of Variance *Birth Weight Body Compos...,1,"[Johnson W , Choh AC , Soloway LE , Czerwin..."
1,1,Increased live birth rate in twin pregnancies ...,Obstet Gynecol,Miro F and Vidal E and Balasch J,Missing,OBJECTIVE: To estimate why the live birth rate...,Embryo Loss/epidemiology *Embryonic Developmen...,1,"[Miro F , Vidal E , Balasch J]"
2,2,Beef in an Optimal Lean Diet study: effects on...,Am J Clin Nutr,Roussell MA and Hill AM and Gaugler TL and Wes...,Missing,BACKGROUND: A Step I diet with lean beef compa...,Animals Apolipoproteins/*blood C-Reactive Prot...,1,"[Roussell MA , Hill AM , Gaugler TL , West ..."
3,3,Physical activity and other health-risk behavi...,Am J Prev Med,Kwan MY and Cairney J and Faulkner GE and Pull...,Missing,BACKGROUND: Research consistently demonstrates...,Adolescent Adult Age Factors Alcohol Drinking/...,1,"[Kwan MY , Cairney J , Faulkner GE , Pullen..."
4,4,Is illicit drug use harmful to cognitive funct...,Am J Epidemiol,Dregan A and Gulliford MC,Missing,"From March to July of 2011, the authors invest...",Adult Cognition/*drug effects Humans Middle Ag...,1,"[Dregan A , Gulliford MC]"
5,5,Physical activity and performance at school: a...,Arch Pediatr Adolesc Med,Singh A and Uijtdewilligen L and Twisk JW and ...,vumc,OBJECTIVE: To describe the prospective relatio...,*Achievement Adolescent Child *Cognition Exerc...,1,"[Singh A , Uijtdewilligen L , Twisk JW , va..."
6,6,Isoflavone intake and risk of gastric cancer: ...,Am J Clin Nutr,Hara A and Sasazuki S and Inoue M and Iwasaki ...,Missing,BACKGROUND: Isoflavones are structurally simil...,Aged *Diet Diet Surveys Estrogen Replacement T...,1,"[Hara A , Sasazuki S , Inoue M , Iwasaki M ..."
7,7,Carotid intimal-medial thickness in active pro...,Am J Cardiol,Hurst RT and Nelson MR and Kendall CB and Cha ...,mayo,Risk of cardiovascular disease and death in re...,Adult Cardiovascular Diseases/*epidemiology/et...,1,"[Hurst RT , Nelson MR , Kendall CB , Cha SS..."
8,8,Does minimum pricing reduce alcohol consumptio...,Addiction,Stockwell T and Auld MC and Zhao J and Martin G,Missing,AIMS: Minimum alcohol prices in British Columb...,Adolescent Adult Aged Alcohol Drinking/*econom...,1,"[Stockwell T , Auld MC , Zhao J , Martin G]"
9,9,Bariatric surgery and long-term cardiovascular...,JAMA,Sjostrom L and Peltonen M and Jacobson P and S...,Missing,CONTEXT: Obesity is a risk factor for cardiova...,Adult *Bariatric Surgery Cardiovascular Diseas...,1,"[Sjostrom L , Peltonen M , Jacobson P , Sjo..."


In [244]:
data['Authors']=map(lambda x: x.replace(" ",""),list(data['Author']))

AttributeError: 'list' object has no attribute 'replace'

In [166]:
sum(data['label'])

1389

In [ ]:
CountVectorizer(vocabulary=['hot', 'cold', 'old'])

In [270]:
allauthors=reduce(lambda x, y: x+y,  data['Author'])

In [271]:
allauthors=set(allauthors)

In [272]:
sorted(allauthors)

["'t Mannetje A",
 '-Olov Adami H',
 'A G',
 'Aadahl M',
 'Aaltonen LA',
 'Aan Het Rot M',
 'Aapro M',
 'Aarabi M',
 'Aarnio K',
 'Aarnio PT',
 'Aarnoutse R',
 'Aarnoutse RE',
 'Aaronson NK',
 'Aarsland D',
 'Aarts MJ',
 'Aarts RM',
 'Aartsen E',
 'Aas E',
 'Aasmaa A',
 'Aavitsland P',
 'Abaan OD',
 'Abacioglu U',
 'Abacioglu UM',
 'Abad FJ',
 'Abad-Vivero EN',
 'Abali R',
 'Aban IB',
 'Abankwa D',
 'Abaryan Z',
 'Abas M',
 'Abassora M',
 'Abaza R',
 'Abbadessa G',
 'Abbafati C',
 'Abbara S',
 'Abbas H',
 'Abbas L',
 'Abbas R',
 'Abbasi F',
 'Abbasi SA',
 'Abbaszadeh B',
 'Abbate A',
 'Abbate M',
 'Abbate R',
 'Abbatini F',
 'Abbey R',
 'Abbink P',
 'Abbink TE',
 'Abbo ED',
 'Abbo LM',
 'Abbott A',
 'Abbott AL',
 'Abbott C',
 'Abbott DE',
 'Abbott DF',
 'Abbott EE',
 'Abbott J',
 'Abbott JD',
 'Abbott JH',
 'Abbott JK',
 'Abbott RA',
 'Abbotts R',
 'Abboud HE',
 'Abboud J',
 'Abboud RT',
 'Abbruzzese E',
 'Abbruzzese JL',
 'Abd-Alazeez M',
 'Abd-Allah F',
 'Abdalla S',
 'Abdallah MS',


In [273]:
len(allauthors)

74260

In [246]:
data1=data.reset_index()

In [249]:
from collections import defaultdict
author_pub=defaultdict(int)

In [250]:
for i in range(1389):
    for name in data1.loc[i]['Author']:
        author_pub[name]+=1

In [252]:
good_authors=[]
for name in author_pub.keys():
    if author_pub[name]>=2:
        good_authors.append(name)

In [437]:
len(good_authors)

829

In [255]:
author_edge=[]

In [256]:
for x in range(12565):
    for i in data1.loc[x]['Author']:
        for j in data1.loc[x]['Author']:
            if i!=j:
                author_edge.append([i,j])

In [257]:
author_edge.sort()

In [268]:
for x in author_edge:
    x=x.sort()

In [260]:
cd /Users/tianhaolu/Documents/Playground/

/Users/tianhaolu/Documents/Playground


In [447]:
csvfile = "/Users/tianhaolu/Documents/Playground/reuter_author_node_reduced.csv"

In [365]:
csvfile = "/Users/tianhaolu/Documents/Playground/reuter_author_edge_reduced.csv"

In [262]:
import csv

In [448]:
good_author_dist=[]
for s in allauthors:
    if author_shortdist[s]<=1:
        good_author_dist.append([s,author_shortdist[s]])

In [446]:
with open("reuter_author_node.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(author_dist)

In [449]:
with open("reuter_author_node_reduced.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(good_author_dist)

In [420]:
elite_edge=[]
bad_authors=allauthors.difference(good_authors).difference(d_1_authors)

In [421]:
for e in author_edge:
    if (e[0] in good_authors) or (e[1] in good_authors):
        elite_edge.append(e)

In [376]:
len(author_edge)

1807760

In [422]:
len(elite_edge)

159026

In [423]:
with open("reuter_author_edge_reduced.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(elite_edge)

In [274]:
author_shortdist=defaultdict(int)

In [275]:
import networkx as nx

In [279]:
author_edge[0]

["'t Mannetje A", 'Ahrens W']

In [277]:
G.add_nodes_from(allauthors)

In [280]:
G.add_edges_from(author_edge)

In [450]:
G.to_undirected()

In [440]:
for s in allauthors:
    if s in good_authors:
        author_shortdist[s]=0
    ##else:
        ##author_shortdist[s]=100000

In [314]:
##very expensive method to calculate dist to good authors. AVOID!!!
##for s in [allauthors]:
##    if s not in good_authors:
##        for t in good_authors:
##            if nx.has_path(G, s, t):
##                s_t=nx.shortest_path_length(G,source=s,target=t)
##                if s_t<author_shortdist[t]:
##                    author_shortdist[s]=s_t
##               if s_t==1:
##              break

In [321]:
## find all dist=1 authors
d_1_authors=[]
for s in allauthors:
    if s not in good_authors:
        for x in nx.all_neighbors(G, s):
            if x in good_authors:
                author_shortdist[s]=1
                d_1_authors.append(s)
                break

In [343]:
d1=d_1_authors

In [442]:
len(allauthors.difference(d_1_authors))

56534

In [334]:
d_2_authors=[]
rest=allauthors.difference(good_authors).difference(d_1_authors)
for s in rest:
    for x in nx.all_neighbors(G, s):
        if x in d_1_authors:
            author_shortdist[s]=2
            d_2_authors.append(s)
            break

In [344]:
d2=d_2_authors

In [337]:
d_3_authors=[]
rest=allauthors.difference(good_authors).difference(d_1_authors).difference(d_2_authors)
for s in rest:
    for x in nx.all_neighbors(G, s):
        if x in d_2_authors:
            author_shortdist[s]=3
            d_3_authors.append(s)
            break

In [345]:
d3=d_3_authors

In [346]:
d_4_authors=[]
rest=allauthors.difference(good_authors).difference(d_1_authors).difference(d_2_authors).difference(d_3_authors)
for s in rest:
    for x in nx.all_neighbors(G, s):
        if x in d_3_authors:
            author_shortdist[s]=4
            d_4_authors.append(s)
            break

In [347]:
d4=d_4_authors

In [350]:
d_5_authors=[]
for s in allauthors.difference(good_authors).difference(d_1_authors).difference(d_2_authors).difference(d_3_authors).difference(d_4_authors):
    for x in nx.all_neighbors(G, s):
        if x in d_4_authors:
            author_shortdist[s]=5
            d_5_authors.append(s)
            break

In [351]:
d5=d_5_authors

In [439]:
len(good_authors)

829

In [330]:
len(d_1_authors)

17726

In [335]:
len(d_2_authors)

26781

In [341]:
len(d_3_authors)

12499

In [348]:
len(d_4_authors)

2654

In [352]:
len(d_5_authors)

406

In [354]:
min(author_shortdist.values())

0

In [340]:
set(good_authors).intersection(set(d_1_authors))

set()

In [326]:
len(allauthors.difference(d_1_authors).difference(d_2_authors))

29753

In [363]:
74260-(len(good_authors)+17726+26781+12499+2654+406)

13365

In [469]:
## new feature
data1

,index,1,2,3,4,5,6,label,Author,Erdos,avg_Erdos
0,0,Eighty-year trends in infant weight and length...,J Pediatr,Johnson W and Choh AC and Soloway LE and Czerw...,Missing,OBJECTIVES: To investigate secular trends in w...,Analysis of Variance *Birth Weight Body Compos...,1,"[Johnson W, Choh AC, Soloway LE, Czerwinski SA...",2,2.833333
1,1,Increased live birth rate in twin pregnancies ...,Obstet Gynecol,Miro F and Vidal E and Balasch J,Missing,OBJECTIVE: To estimate why the live birth rate...,Embryo Loss/epidemiology *Embryonic Developmen...,1,"[Miro F, Vidal E, Balasch J]",2,2.666667
2,2,Beef in an Optimal Lean Diet study: effects on...,Am J Clin Nutr,Roussell MA and Hill AM and Gaugler TL and Wes...,Missing,BACKGROUND: A Step I diet with lean beef compa...,Animals Apolipoproteins/*blood C-Reactive Prot...,1,"[Roussell MA, Hill AM, Gaugler TL, West SG, He...",2,2.875000
3,3,Physical activity and other health-risk behavi...,Am J Prev Med,Kwan MY and Cairney J and Faulkner GE and Pull...,Missing,BACKGROUND: Research consistently demonstrates...,Adolescent Adult Age Factors Alcohol Drinking/...,1,"[Kwan MY, Cairney J, Faulkner GE, Pullenayegum...",3,3.750000
4,4,Is illicit drug use harmful to cognitive funct...,Am J Epidemiol,Dregan A and Gulliford MC,Missing,"From March to July of 2011, the authors invest...",Adult Cognition/*drug effects Humans Middle Ag...,1,"[Dregan A, Gulliford MC]",2,2.000000
5,5,Physical activity and performance at school: a...,Arch Pediatr Adolesc Med,Singh A and Uijtdewilligen L and Twisk JW and ...,vumc,OBJECTIVE: To describe the prospective relatio...,*Achievement Adolescent Child *Cognition Exerc...,1,"[Singh A, Uijtdewilligen L, Twisk JW, van Mech...",0,0.800000
6,6,Isoflavone intake and risk of gastric cancer: ...,Am J Clin Nutr,Hara A and Sasazuki S and Inoue M and Iwasaki ...,Missing,BACKGROUND: Isoflavones are structurally simil...,Aged *Diet Diet Surveys Estrogen Replacement T...,1,"[Hara A, Sasazuki S, Inoue M, Iwasaki M, Shima...",0,0.875000
7,7,Carotid intimal-medial thickness in active pro...,Am J Cardiol,Hurst RT and Nelson MR and Kendall CB and Cha ...,mayo,Risk of cardiovascular disease and death in re...,Adult Cardiovascular Diseases/*epidemiology/et...,1,"[Hurst RT, Nelson MR, Kendall CB, Cha SS, Ress...",2,2.833333
8,8,Does minimum pricing reduce alcohol consumptio...,Addiction,Stockwell T and Auld MC and Zhao J and Martin G,Missing,AIMS: Minimum alcohol prices in British Columb...,Adolescent Adult Aged Alcohol Drinking/*econom...,1,"[Stockwell T, Auld MC, Zhao J, Martin G]",0,0.250000
9,9,Bariatric surgery and long-term cardiovascular...,JAMA,Sjostrom L and Peltonen M and Jacobson P and S...,Missing,CONTEXT: Obesity is a risk factor for cardiova...,Adult *Bariatric Surgery Cardiovascular Diseas...,1,"[Sjostrom L, Peltonen M, Jacobson P, Sjostrom ...",1,1.857143


In [466]:
def dist_to_good_author(authors):
    return min(map(lambda x:author_shortdist[x] ,authors))

data1['Erdos']=map(lambda x: dist_to_good_author(x), data1['Author'])

In [468]:
def avg_to_good_author(authors):
    return np.mean(map(lambda x:author_shortdist[x] ,authors))

data1['avg_Erdos']=map(lambda x: avg_to_good_author(x), data1['Author'])

In [486]:
np.mean(data1[data1['label']==0]['Erdos'])

12.673496778811739

In [487]:
np.mean(data1[data1['label']==1]['Erdos'])

3.637149028077754

In [490]:
data1['Erdos'].corr(data1['avg_Erdos'])

0.99641060321593755

In [474]:
data1.shape

(12565, 11)

In [465]:
for x in allauthors:
    if author_shortdist[x]==100000:
        if author_pub[x]==1:
            author_shortdist[x]=20
        else:
            author_shortdist[x]=50

In [485]:
data1.to_csv('reuters_erdos.csv')

In [1]:
cd /Users/tianhaolu/Documents/Playground

/Users/tianhaolu/Documents/Playground


In [7]:
data2=pd.read_csv("reuters_erdos.csv")

In [29]:
len(data2[data2['College']=='Missing'])

9325

In [42]:
t=map(lambda x:x.split("/"),data2['Ingredients'].loc[1].split("*"))

In [46]:
set(map(lambda x:x.rstrip(), chain.from_iterable(t)))

{'',
 'Embryo Loss',
 'Embryonic Development Female Humans',
 'Live Birth Maternal Age Pregnancy Pregnancy, Twin',
 'Reproductive Techniques, Assisted Retrospective Studies Spain',
 'epidemiology',
 'physiology',
 'statistics & numerical data'}

In [56]:
ingredients=[map(lambda x:x.split("/"), data2['Ingredients'])]

In [58]:
ingredients

[[['Analysis of Variance *Birth Weight Body Composition *Body Height Body Mass Index Child Development',
   '*physiology Cohort Studies Female Follow-Up Studies Growth and Development',
   '*physiology Humans Infant Infant, Newborn Linear Models Longitudinal Studies Male Risk Factors Sex Factors Time Factors'],
  ['Embryo Loss',
   'epidemiology *Embryonic Development Female Humans *Live Birth Maternal Age Pregnancy Pregnancy, Twin',
   '*physiology',
   'statistics & numerical data *Reproductive Techniques, Assisted Retrospective Studies Spain',
   'epidemiology'],
  ['Animals Apolipoproteins',
   '*blood C-Reactive Protein',
   '*metabolism Cattle Cholesterol',
   '*blood Cholesterol, LDL',
   'blood *Diet, Fat-Restricted Dietary Fats',
   'administration & dosage Dietary Proteins',
   'administration & dosage Female Humans Hypercholesterolemia',
   'blood',
   '*diet therapy Male *Meat Middle Aged'],
  ['Adolescent Adult Age Factors Alcohol Drinking',
   '*epidemiology Canada Child 

In [78]:
ingredients2=[]
for i in ingredients[0]:  ##double nested...
    j=map(lambda x:x.split("*"),i)
    j=chain.from_iterable(j)
    j=map(lambda x:x.rstrip(),j)
    j=set(j)
    if '' in j:
        j.remove('')
    ingredients2.append(j)

In [81]:
data2['ingredients']=ingredients2

In [96]:
data2.loc[1503]['ingredients']

{'Adolescent Age of Onset B-Lymphocytes',
 'administration & dosage',
 'administration & dosage Cytokines',
 'administration & dosage Immunity, Cellular',
 'administration & dosage Male Prevalence Prospective Studies Treatment Outcome Vitamins',
 'blood Child Cholecalciferol',
 'blood Diabetes Mellitus, Type 1',
 'drug effects',
 'drug therapy',
 'epidemiology',
 'epidemiology C-Peptide',
 'immunology Dose-Response Relationship, Drug Double-Blind Method Drug Therapy, Combination Female Humans Hypoglycemic Agents',
 'immunology Insulin',
 'physiology Body Mass Index Brazil'}

In [95]:
data2.loc[1503]['Ingredients']

'Adolescent Age of Onset B-Lymphocytes/drug effects/*physiology Body Mass Index Brazil/epidemiology C-Peptide/blood Child Cholecalciferol/*administration & dosage Cytokines/blood Diabetes Mellitus, Type 1/*drug therapy/epidemiology/immunology Dose-Response Relationship, Drug Double-Blind Method Drug Therapy, Combination Female Humans Hypoglycemic Agents/administration & dosage Immunity, Cellular/*drug effects/immunology Insulin/*administration & dosage Male Prevalence Prospective Studies Treatment Outcome Vitamins/administration & dosage'

In [75]:
a={'','b','12'}

In [76]:
a.remove('')

In [77]:
a

{'12', 'b'}

In [39]:
list(chain.from_iterable([[180.0,1,2,"SE"], [173.8], [164.2], [156.5], [147.2], [138.2]]))

[180.0, 1, 2, 'SE', 173.8, 164.2, 156.5, 147.2, 138.2]

In [ ]:
data3=pd.read_csv("reuters_erdos.csv")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [12]:
logit = LogisticRegression()

In [18]:
X=data3['avg_Erdos']
y=data3['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [34]:
sum(X_train)

107529.25729088331

In [33]:
sum(y_train)

985

In [26]:
X_train.reshape((8795,1)) 

array([[  2.28571429],
       [  2.8       ],
       [  3.75      ],
       ..., 
       [  0.97727273],
       [ 50.        ],
       [ 50.        ]])

In [27]:
logit.fit(X_train.reshape((8795,1)) , y_train.reshape((8795,1)) )

/Users/tianhaolu/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [30]:
len(X_test)

3770

In [32]:
predicted = logit.predict(X_test.reshape((3770,1)))


0

In [36]:
probs = logit.predict_proba(X_test.reshape((3770,1)))
print probs

[[ 0.86311988  0.13688012]
 [ 0.86720868  0.13279132]
 [ 0.86160698  0.13839302]
 ..., 
 [ 0.85815163  0.14184837]
 [ 0.85853909  0.14146091]
 [ 0.86274296  0.13725704]]


In [50]:
pred=[0]*len(X_test)

In [51]:
for i in range(len(probs)):
    if probs[i][1]>0.14:
        pred[i]=1
    else:
        pred[i]=0

In [55]:
y_test

10095    0
1624     0
3975     0
9004     0
2476     0
1783     0
12014    0
10665    0
823      1
406      1
6073     0
7053     0
3699     0
930      1
1494     0
5875     0
4030     0
3481     0
663      1
12186    0
4894     0
1425     0
9535     0
8606     0
3380     0
10186    0
8391     0
6346     0
8703     0
11486    0
        ..
6339     0
2192     0
617      1
8111     0
8223     0
6010     0
12038    0
11243    0
7580     0
8252     0
4067     0
11124    0
585      1
3755     0
12255    0
1453     0
9024     0
2862     0
11881    0
3810     0
9617     0
11597    0
11202    0
5438     0
5706     0
5686     0
11582    0
2639     0
11879    0
6652     0
Name: label, dtype: int64

In [56]:
good_pred=0
for i in range(len(pred)):
    if y_test.tolist()[i]==pred[i]:
        good_pred+=1

In [59]:
float(good_pred)/len(pred)

0.8106100795755968

In [ ]:
D
y=D['label']
X=D.drop('label',1)

clf = linear_model.LogisticRegression(penalty='l1',C=1e5)
clf.fit(X, y)